In [38]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
%autoreload 2

In [40]:
from master import *
from models import *
from torchsummary import summary

In [41]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
DATASET='large'
ROOT_DIR = f'/beegfs/bva212/fma_{DATASET}_cqt/'
PATH_FILE = 'checksums'
BATCH_SIZE = 32
VAL_SPLIT = 0.2
TRANSFORM_PROB  = 0.5
SR = 22050
MODEL_SAVE_PATH = '/beegfs/sc6957/capstone/models/'
MODE = 'jigsaw'
MODEL_ARCHITECTURE = 'snet2'  #['l3net','snet',snet2',anet']
SEPARABLE = False
REDUCE_TWO_CLASS = True #Applies to jigsaw (6-class to 2-class)
NUM_SECONDS=10
TRANSFORM_DATA = False

In [42]:
start = time.time()
MODEL,MODEL_NAME = load_model(DEVICE, MODEL_ARCHITECTURE, MODE, SEPARABLE)
master = Master(DEVICE, ROOT_DIR, PATH_FILE, SR, BATCH_SIZE, VAL_SPLIT, TRANSFORM_PROB, REDUCE_TWO_CLASS, MODE, MODEL, NUM_SECONDS, TRANSFORM_DATA)
print(f'time for instantiating audio net object - {time.time() - start}')

Instantiating Model snet2_jigsaw
Number of Seconds: 10
MODE: jigsaw
Getting Train & Validation Datasets
Excluding these 3 files - ['098/098567.mp3_cqt.npy', '098/098565.mp3_cqt.npy', '098/098569.mp3_cqt.npy']
There are a total of 106410 music files in the root directory
# Training samples: 85127 	 # Validation samples: 21282
	 --Done
Creating Train & Validation Dataloaders
Length of Train dataloader: 2661 	 Validation dataloader: 666
	 --Done
	 --Done
Init actions done
time for instantiating audio net object - 29.157325267791748


In [43]:
# start = time.time()
# batch = next(iter(master.dataloaders['train']))
# print(f'time for fetching one batch - {time.time() - start}')

In [44]:
master.model

snet2_jigsaw(
  (conv_layers): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(2, 2), padding=(5, 5))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2), padding=(5, 5))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(5, 5))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): AvgPool2d(kernel_size=3, stride=3, padding=0)
    (10): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1), padding=(4, 4))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): Conv2d(128, 256, kernel_size=(4, 4), stride=(1, 1), padding=(4, 4))
    (14): BatchNorm2d(256, eps=1e-05, momentum=0.1, 

In [45]:
master.input_size

torch.Size([3, 84, 135])

In [46]:
summary(master.model,input_size=master.input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 45, 71]             416
       BatchNorm2d-2           [-1, 16, 45, 71]              32
              ReLU-3           [-1, 16, 45, 71]               0
            Conv2d-4           [-1, 32, 26, 39]          12,832
       BatchNorm2d-5           [-1, 32, 26, 39]              64
              ReLU-6           [-1, 32, 26, 39]               0
            Conv2d-7           [-1, 64, 32, 45]          51,264
       BatchNorm2d-8           [-1, 64, 32, 45]             128
              ReLU-9           [-1, 64, 32, 45]               0
        AvgPool2d-10           [-1, 64, 10, 15]               0
           Conv2d-11          [-1, 128, 15, 20]         131,200
      BatchNorm2d-12          [-1, 128, 15, 20]             256
             ReLU-13          [-1, 128, 15, 20]               0
           Conv2d-14          [-1, 256,

In [47]:
MODEL_SAVE_PATH = MODEL_SAVE_PATH+MODEL_NAME+f'_{DATASET}_'

In [ ]:
master.train(num_epochs= 25, learning_rate=1e-5, print_every=5, verbose=True, save=True, checkpoint_every=5, model_save_path = MODEL_SAVE_PATH)

Saving model at /beegfs/sc6957/capstone/models/snet2_jigsaw_large_
Instantiating Optimzer, Loss Criterion, Scheduler
	 --Done
Training started
Epoch 1/25
----------
